### Imports

In [60]:
import os
from tensorflow import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import layers

In [49]:
def load_dataset():
     # load dataset
     (X_train, y_train), (X_test, y_test) = mnist.load_data()
     # reshape dataset to have a single channel
     X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
     X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

     return (X_train, y_train), (X_test, y_test)

# scales images to the [0,1] range
def scale_images(images):
	# convert from integers to floats
	images_norm = images.astype('float32')
	# normalize to range 0-1
	images_norm = images_norm / 255.0
	# return normalized images
	return images_norm

In [50]:
# load dataset
(X_train, y_train), (X_test, y_test) = load_dataset()
# summarize loaded dataset
print('Train: X=%s, y=%s' % (X_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (X_test.shape, y_test.shape))
X_train = scale_images(X_train)
X_test = scale_images(X_test)

Train: X=(60000, 28, 28, 1), y=(60000,)
Test: X=(10000, 28, 28, 1), y=(10000,)


In [51]:
nmist_class_model = keras.Sequential(
    [
        keras.Input(shape=X_train.shape[1:]),
        layers.Conv2D(32, (3,3), activation="relu", kernel_initializer='he_uniform'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(name='flatten_layer'),
        layers.Dense(10, activation='softmax')
    ]
)
nmist_class_model.summary()

nmist_class_model.compile(
    optimizer=keras.optimizers.Adam(),
    # Loss function to minimize
    loss=keras.losses.CategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.CategoricalAccuracy()],
)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 batch_normalization_8 (Batc  (None, 26, 26, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 batch_normalization_9 (Batc  (None, 11, 11, 64)       256       
 hNormalization)                                                 
                                                      

In [58]:
# split the train set to train & validation
x_train, x_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.15, stratify=None)
print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Val: X=%s, y=%s' % (x_val.shape, y_val.shape))
# one hot encode target values
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
print("Fit model on training data")
history = nmist_class_model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=10,
    validation_data=(x_val, y_val))

Train: X=(51000, 28, 28, 1), y=(51000,)
Val: X=(9000, 28, 28, 1), y=(9000,)
Fit model on training data
Epoch 1/10
797/797 [==============================] - 22s 26ms/step - loss: 0.1549 - categorical_accuracy: 0.9530 - val_loss: 0.0654 - val_categorical_accuracy: 0.9790
Epoch 2/10
797/797 [==============================] - 25s 31ms/step - loss: 0.0525 - categorical_accuracy: 0.9837 - val_loss: 0.0641 - val_categorical_accuracy: 0.9819
Epoch 3/10
797/797 [==============================] - 25s 31ms/step - loss: 0.0356 - categorical_accuracy: 0.9890 - val_loss: 0.0468 - val_categorical_accuracy: 0.9854
Epoch 4/10
797/797 [==============================] - 25s 31ms/step - loss: 0.0247 - categorical_accuracy: 0.9921 - val_loss: 0.0502 - val_categorical_accuracy: 0.9864
Epoch 5/10
797/797 [==============================] - 25s 32ms/step - loss: 0.0216 - categorical_accuracy: 0.9933 - val_loss: 0.0500 - val_categorical_accuracy: 0.9876
Epoch 6/10
797/797 [==============================] - 25s

In [59]:
# Evaluate the model on the test data
print("Evaluate on test data")
results = nmist_class_model.evaluate(X_test, to_categorical(y_test), batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
79/79 [==============================] - 1s 12ms/step - loss: 0.0627 - categorical_accuracy: 0.9863
test loss, test acc: [0.06267518550157547, 0.986299991607666]


In [61]:
# save the model
model_checkpoint_dir = "c:/model_checkpoints/"
if os.path.isdir(model_checkpoint_dir) is not True:
    os.mkdir(model_checkpoint_dir)
model_path = os.path.join(model_checkpoint_dir,'mnist_classification_convnet')
nmist_class_model.save(model_path)

INFO:tensorflow:Assets written to: c:/model_checkpoints/mnist_classification_convnet\assets


INFO:tensorflow:Assets written to: c:/model_checkpoints/mnist_classification_convnet\assets
